# DenseNet121 - 64x64

### Setup

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = "3"

In [2]:
import gc
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split

In [3]:
print("CUDA avilable:",
     True if tf.config.list_physical_devices("GPU") else False)

CUDA avilable: True


In [4]:
SEED = 1234
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [5]:
input_shape = 64

### Preprocessing

In [6]:
PATH = "C:/Users/shj03/Desktop/git_files/ANBW/dataset"
train = pd.read_csv(f"{PATH}/train.csv")
test = pd.read_csv(f"{PATH}/test.csv")
submission = pd.read_csv(f"{PATH}/submission.csv")

In [7]:
x_train = train.drop(["id", "digit", "letter"], axis=1).values
x_train = x_train.reshape(-1, 28, 28, 1)
x_train = np.where((x_train<=20)&(x_train!=0) ,0.,x_train)
x_train = x_train/255
x_train = x_train.astype("float32")
x_train_resized = np.zeros([2048, input_shape, input_shape, 3], dtype=np.float32)

for idx, img in enumerate(x_train):
    converted = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    resized = cv2.resize(converted, (input_shape, input_shape), interpolation=cv2.INTER_CUBIC)
    del converted
    x_train_resized[idx] = resized
    del resized
    gc.collect()

x_train_resized.shape

(2048, 64, 64, 3)

In [8]:
y = train["digit"]
y_train = np.zeros((len(y), len(y.unique())))

for idx, digit in enumerate(y):
    y_train[idx, digit] = 1

y_train.shape

(2048, 10)

In [9]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train_resized, y_train, test_size=0.2, random_state=SEED, shuffle=True)

### Initialing model

In [10]:
from tensorflow.keras.callbacks import EarlyStopping

In [11]:
BATCH = 32
EPOCHS = 100
CALLBACKS = [EarlyStopping(monitor="val_loss", verbose=1, patience=20)]

In [12]:
densenet = tf.keras.applications.densenet.DenseNet121(
    weights=None,
    input_shape=(64, 64, 3),
    classes=10,
)

In [13]:
model = tf.keras.Sequential()
model.add(densenet)

model.compile(loss="categorical_crossentropy",
             optimizer="Adam",
             metrics=["accuracy"])

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Functional)     (None, 10)                7047754   
Total params: 7,047,754
Trainable params: 6,964,106
Non-trainable params: 83,648
_________________________________________________________________


### Train model

In [ ]:
history = model.fit(x_train, y_train, batch_size=BATCH, epochs=EPOCHS, callbacks=CALLBACKS, validation_data=(x_valid, y_valid))

Epoch 1/100
52/52 [==============================] - 11s 203ms/step - loss: 1.8203 - accuracy: 0.3455 - val_loss: 2.6572 - val_accuracy: 0.0854
Epoch 2/100
52/52 [==============================] - 5s 94ms/step - loss: 1.1709 - accuracy: 0.6099 - val_loss: 3.0597 - val_accuracy: 0.0854
Epoch 3/100
52/52 [==============================] - 5s 92ms/step - loss: 0.8188 - accuracy: 0.7247 - val_loss: 3.2633 - val_accuracy: 0.0829
Epoch 4/100
52/52 [==============================] - 5s 97ms/step - loss: 0.6204 - accuracy: 0.7821 - val_loss: 5.8740 - val_accuracy: 0.0854
Epoch 5/100
52/52 [==============================] - 5s 94ms/step - loss: 0.5212 - accuracy: 0.8168 - val_loss: 3.1035 - val_accuracy: 0.1171
Epoch 6/100
52/52 [==============================] - 5s 93ms/step - loss: 0.3625 - accuracy: 0.8767 - val_loss: 2.8065 - val_accuracy: 0.1512
Epoch 7/100
52/52 [==============================] - 5s 93ms/step - loss: 0.3795 - accuracy: 0.8657 - val_loss: 2.3928 - val_accuracy: 0.3610
Epoc

### Predict

In [ ]:
x_test = test.drop(["id", "letter"], axis=1).values
x_test = x_test.reshape(-1, 28, 28, 1)
x_test = np.where((x_test<=20)&(x_test!=0), 0., x_test)
x_test = x_test/255
x_test = x_test.astype("float32")
x_test_resized = np.zeros([20480, input_shape, input_shape, 3], dtype=np.float32)

for idx, img in enumerate(x_test):
    converted = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    resized = cv2.resize(converted, (input_shape, input_shape), interpolation=cv2.INTER_CUBIC)
    del converted
    x_test_resized[idx] = resized
    del resized
    gc.collect()

x_test_resized.shape

In [ ]:
submission['digit'] = np.argmax(model.predict(x_test_resized), axis=1)
submission.head()

In [ ]:
submission.to_csv('DenseNet121-64x64.csv', index=False)